# IFT3395/6390: Devoir 1

Mahmoud Abou-Nassif C2840

Jean Archambault

---

## 1 Petit exercice de probabilités

On modélise le problème de la façon suivante:

$$ X_1 \in \{P, N\} \quad et \quad X_2 \in \{C, B\}$$

Où  


$P$ : Test positif  
$N$ : Test negatif  
$C$ : Cancer  
$B$ : Benin  

D'après l'énoncé, on a déja les egalités suivantes:

- $ P(X_2=C)=0.01 $  
- $ P(X_1=P| X_2=C)=0.8 $
- $ P(X_1=P| X_2=B)=0.096 $  

La question consiste à trouver la probabilité suivante: 

$$ P(X_2=C|X_1=P)$$

On peut déduire la probabilité de l'absence d'un cancer du sein chez les femmes dans la quarantaine participant à un test de routine:    

$$ P(X_2=B) = 1 - P(X_2=C) = 0.99 $$

D'après la règle de Bayes on a:

$$ 
\begin{align}
P(X_2=C|X_1=P) & = \frac{P(X_1=P|X_2=C)P(X_2=C)}{P(X_1=P)} \\   
& = \frac{P(X_1=P|X_2=C)P(X_2=C)}{P(X_1=P|X_2=C)P(X_2=C) + P(X_1=P|X_2=B)P(X_2=B)}\approx 7.76\% 
\end{align} 
$$


On déduit que la réponse des médecins est incorrecte. La probabilité est moins de 10% donc réponse $F)$

## 3 Partie pratique: estimation de densité